<center>
<img src="https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/img/this_is_data_anal.png" width="200" height="200"><br>
</center>

## 5.2 구매 데이터를 분석하여 상품 추천하기

이번 절에서는 구매 데이터 분석에 기반한 온라인 스토어 상품 추천 시뮬레이션 예제에 대해 알아보자.
예제에서 피처 엔지니어링, 그리고 행렬 완성 기반 점수 예측 방법을 이용하여 상품 추천 시뮬레이션을 수행한다. 분석에 사용할 데이터는 영국의 한 선물 판매 온라인 스토어에서 발생한 거래 데이터로, 주 고객은 선물 도매상이다.

### step.1 탐색적 분석: UK Retail 데이터 분석하기

예제에서 사용할 UK Retail 데이터셋은 다음과 같은 피처로 구성되어 있다.

- InvoiceNo : 거래 고유 번호
- StockCode : 상품 고유 번호
- Description : 상품명
- Quantity : 거래 수량
- InvoiceDate : 거래 일시
- UnitPrice : 상품 단가
- CustomerID : 구매자 고유 번호
- Country : 구매 국가

아래 코드를 실행해 보면 약 54만 개 정도의 데이터가 존재하고 그 중 14만 개의 데이터는 구매자 정보가 결측값인 것을 알 수 있다.

In [11]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

- 데이터셋 살펴보기

In [12]:
# 영국 온라인 스토어 도매 거래 데이터
df = pd.read_csv('/home/jaeyoon89/python-data-analysis/data/online_retail.csv', dtype={'CustomerID':str,'InvoicedID':str},encoding="ISO-8859-1")
df['InvoicedDate'] = pd.to_datetime(df['InvoiceDate'], format="%m/%d/%Y %H:%M")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     541909 non-null  object        
 1   StockCode     541909 non-null  object        
 2   Description   540455 non-null  object        
 3   Quantity      541909 non-null  int64         
 4   InvoiceDate   541909 non-null  object        
 5   UnitPrice     541909 non-null  float64       
 6   CustomerID    406829 non-null  object        
 7   Country       541909 non-null  object        
 8   InvoicedDate  541909 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 37.2+ MB
None


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoicedDate
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01 08:26:00
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01 08:26:00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00


본격적인 탐색적 데이터 분석에 앞서, 데이터에서 예외적인 상황을 필터링하여 이상치를 제거해야 한다. 가장 먼저 결측 데이터를 제거하자. 다음 코드 결과는 유저정보가 없는 13만 5천여 개의 데이터, 상품 상세정보가 없는 1500여 개의 데이터를 제거한 것이다.

- 결측 데이터 제거하기

In [13]:
df.isnull().sum()

InvoiceNo            0
StockCode            0
Description       1454
Quantity             0
InvoiceDate          0
UnitPrice            0
CustomerID      135080
Country              0
InvoicedDate         0
dtype: int64

In [14]:
df = df.dropna()
print(df.shape)

(406829, 9)


다음은 데이터가 일반적이지 않은 경우를 탐색하고 이를 제거하자. 이번에는 상품 수량 데이터가 이상한 경우를 탐색해보자. 아래의 코드는 상품수량이 0 이하인 경우 해당 값을 데이터 프레임에서 제거하는 과정이다. 

- 탐색 데이터의 조건 필터링 : 상품 수량이 0 이하인 경우

In [15]:
# 상품 수량이 음수인 경우를 제거한다.
print(df[df['Quantity']<=0].shape[0])
df = df[df['Quantity']>0]


8905


이번엔 상품 가격이 0 이하인 경우를 탐색하자. 역시 같은 방법으로 제거해보자.

- 탐색 데이터의 조건 필터링 : 상품가격이 0 이하인 경우

In [19]:
# 상품 가격이 0 이하인 경우를 제거한다.
print(df[df['UnitPrice']<=0].shape[0])
df = df[df['UnitPrice']>0]

40


마지막으로 상품 코드가 이상한 경우를 탐색하고 제거하자. 데이터 내의 StockCode를 관찰해보면 대부분의 상품 코드가 번호로 이루어져 있는 것을 알 수 있다. 따라서 상품 코드가 번호가 아닌 경우는 예외적인 상황일 것이다.

- 탐색 데이터의 조건 필터링 : 상품 코드가 일반적이지 않은 경우

In [21]:
# 상품 코드가 일반적이지 않은 경우를 탐색한다.
df['ContainDigit'] = df['StockCode'].apply(lambda x: any(c.isdigit() for c in x))
print(df[df['ContainDigit'] == False].shape[0])
df[df['ContainDigit'] == False].head()

1414


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoicedDate,ContainDigit
45,536370,POST,POSTAGE,3,12/1/2010 8:45,18.00,12583,France,2010-12-01 08:45:00,False
386,536403,POST,POSTAGE,1,12/1/2010 11:27,15.00,12791,Netherlands,2010-12-01 11:27:00,False
1123,536527,POST,POSTAGE,1,12/1/2010 13:04,18.00,12662,Germany,2010-12-01 13:04:00,False
2239,536569,M,Manual,1,12/1/2010 15:35,1.25,16274,United Kingdom,2010-12-01 15:35:00,False
2250,536569,M,Manual,1,12/1/2010 15:35,18.95,16274,United Kingdom,2010-12-01 15:35:00,False


In [22]:
# 상품 코드가 일반적이지 않은 경우를 제거한다.
df = df[df['ContainDigit'] == True]

본격적으로 탐색적 분석을 수행할 차례이다. 우리는 어떤 방향성을 가지고 탐색적 분석을 해야할까? 그러려면 우리가 온라인 스토어의 운영자 입장이 되어 생각을 해보자.
연말에 방문한 유저들에게 상품을 추천해준다는 것은 유저-상품 간의 구매 확률을 예측해보는 시뮬레이션이라고 할 수 있다. 이러한 예측 시뮬레이션은 아래와 같은 분석 과정이 필요하다.